In [1]:
%reload_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
import src.config as config

from datetime import datetime

from src.inference import (
    get_feature_store, 
    load_batch_of_features_for_inference,
    load_model_from_registry,
    get_model_predictions,
    )

backfill_days = 120

In [8]:
current_date = datetime.now().date()

data = load_batch_of_features_for_inference(current_date, backfill_days=backfill_days)
display(data.head())
display(data.tail())

2025-08-17 18:18:39,358 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-17 18:18:39,381 INFO: Initializing external client
2025-08-17 18:18:39,382 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-17 18:18:40,248 WARNING: UserWarning: The installed hopsworks client version 4.1.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-08-17 18:18:41,067 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1051798
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.96s) 


,datetime,demand,ba_code
9073,2024-04-05,51031.0,AECI
7522,2024-04-06,48781.0,AECI
11341,2024-04-07,45064.0,AECI
10989,2024-04-08,44801.0,AECI
13337,2024-04-09,42950.0,AECI


,datetime,demand,ba_code
31161,2025-08-11,2284.0,WAUW
30147,2025-08-12,2473.0,WAUW
28804,2025-08-13,2588.0,WAUW
31042,2025-08-14,2431.0,WAUW
30196,2025-08-15,2349.0,WAUW


In [9]:
model, preprocessing_pipeline = load_model_from_registry()

2025-08-17 18:18:58,503 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-17 18:18:58,506 INFO: Initializing external client
2025-08-17 18:18:58,506 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-17 18:18:58,909 WARNING: UserWarning: The installed hopsworks client version 4.1.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-08-17 18:18:59,611 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1051798


In [ ]:
features_end = str(data["datetime"].dt.date.max())
print(features_end)

predictions = get_model_predictions(
    model=model, 
    preprocessing_pipeline=preprocessing_pipeline, 
    X=data, 
    features_end=features_end,
    backfill_days=backfill_days,
    )
predictions.head()

2025-08-15


,ba_code,predicted_demand,datetime
0,AECI,58350.0,2025-04-18
1,AECI,56613.0,2025-04-19
2,AECI,56712.0,2025-04-20
3,AECI,55488.0,2025-04-21
4,AECI,58880.0,2025-04-22


In [28]:
feature_store = get_feature_store()

feature_group_predictions = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=config.FEATURE_GROUP_MODEL_PREDICTIONS_VERSION,
    description="Predictions generated by the model",
    primary_key=["ba_code", "datetime"],
    event_time="datetime",
)

2025-08-17 18:33:41,167 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-17 18:33:41,170 INFO: Initializing external client
2025-08-17 18:33:41,170 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-17 18:33:41,556 WARNING: UserWarning: The installed hopsworks client version 4.1.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-08-17 18:33:42,282 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1051798


In [29]:
feature_group_predictions.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 6413/6413 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1051798/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)